In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/data/clean_reviews.csv")
print("✅ Data loaded from Drive:", df.shape)


✅ Data loaded from Drive: (1721, 9)


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()
df['sentiment_score'] = df['clean_text'].apply(lambda x: sia.polarity_scores(x)['compound'])


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
!pip install transformers torch

from transformers import pipeline
sentiment_model = pipeline("sentiment-analysis", device=0)

# Process in chunks to handle big data
batch = df['clean_text'][:2000]
df.loc[:1999, 'sentiment_label'] = [sentiment_model(x[:512])[0]['label'] for x in batch]


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
brand_features = df.groupby('brand').agg({
    'rating': ['mean','count'],
    'sentiment_score': 'mean'
}).reset_index()
brand_features.columns = ['brand','avg_rating','review_count','avg_sentiment']


In [ ]:
user_features = df.groupby('username').agg({
    'rating': ['mean','count'],
    'sentiment_score': 'mean'
}).reset_index()
user_features.columns = ['username','avg_rating','num_reviews','avg_sentiment']


In [ ]:
brand_features.to_csv("/content/drive/MyDrive/data/brand_insights.csv", index=False)
user_features.to_csv("/content/drive/MyDrive/data/customer_insights.csv", index=False)
